# Print Runtime

In [1]:
sf=10

In [2]:
from psutil import *
vCPU = str(cpu_count()) + " vCPU"
mem=round(virtual_memory().total/(1024 * 1024 * 1024),0)
runtime = 'Lineitem-SF'+str(sf)+' ,Google Colab, '+vCPU+' '+str(mem)+'GB'
runtime

'Lineitem-SF10 ,Google Colab, 16 vCPU 59.0GB'

# Install Packages

In [2]:
!pip install pyarrow --upgrade
!pip install duckdb --pre --upgrade
!pip install polars
!pip install datafusion=0.8
!pip install pyspark
!curl https://clickhouse.com/ | sh

  Using cached duckdb-0.7.2.dev1623-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (14.9 MB)
  Using cached polars-0.17.0-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.9/95.9 kB 8.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... error
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [6 lines of output]
      
      Cargo, the Rust package manager, is not installed or is not on PATH.
      This package requires Rust and Cargo to compile extensions. Install it through
      the system's package manager or via https://rustup.rs/
      
      Checking for Rust toolchain....
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered 

#Generate Parquet File

In [4]:
%%time
import duckdb
import pathlib


for x in range(0, sf) :
  con=duckdb.connect()
  con.sql('PRAGMA disable_progress_bar;SET preserve_insertion_order=false')
  con.sql(f"CALL dbgen(sf={sf} , children ={sf}, step = {x})") 
  for tbl in ['lineitem'] :
     pathlib.Path(f'./{tbl}').mkdir(parents=True, exist_ok=True) 
     con.sql(f"COPY (SELECT * FROM {tbl}) TO './{tbl}/{x}.parquet' ")
  con.close()

CPU times: user 5min 3s, sys: 21 s, total: 5min 24s
Wall time: 2min 40s


# DataFusion

In [5]:
%%time
from datafusion import SessionContext
ctx = SessionContext()
ctx.register_parquet('lineitem', './lineitem/*.parquet')
df = ctx.sql("select * from lineitem order by l_shipdate")
df.write_parquet("lineitem_Datafusion")

CPU times: user 2min 12s, sys: 19.7 s, total: 2min 32s
Wall time: 1min 4s


# Spark

In [6]:
%%time
import pyspark as spark
from pyspark.sql import SparkSession
spark=SparkSession.builder.master("local[*]").appName("PySpark Read Parquet").getOrCreate()
spark.read.parquet('./lineitem/*.parquet').createOrReplaceTempView("lineitem")
spark.sql(""" select * from lineitem order by l_shipdate """).write.parquet("lineitem_spark")

JAVA_HOME is not set


RuntimeError: Java gateway process exited before sending its port number

#DuckDB

In [3]:
%%time
import duckdb 
duckdb.sql(
'''
PRAGMA disable_progress_bar;
COPY  (select * from './lineitem/*.parquet' order by l_shipdate )
to 'lineitem_DuckDB' (FORMAT 'PARQUET', PER_THREAD_OUTPUT TRUE)
''')

CPU times: user 3min 49s, sys: 44.7 s, total: 4min 33s
Wall time: 29.6 s


# Clickhouse

In [12]:
%%time
#Query 1
!./clickhouse local --query=" \
    select * FROM file ('./lineitem/*.parquet',Parquet) \
    order by l_shipdate \
    INTO OUTFILE 'clickhouse.parquet' \
    FORMAT Parquet \
    "

CPU times: user 1.61 s, sys: 520 ms, total: 2.13 s
Wall time: 1min 24s


# Polars

In [9]:
%%time
import polars as pl
pl.scan_parquet("./lineitem/*.parquet").sort("l_shipdate").sink_parquet("lineitem_sort_polars.parquet")

STREAMING CHUNK SIZE: 3125 rows


CPU times: user 2min 29s, sys: 18.6 s, total: 2min 48s
Wall time: 1min 6s


# Pyarrow

In [10]:
%%time
import pyarrow.dataset as ds
lineitem = ds.dataset('./lineitem',format="parquet").sort_by('l_shipdate')
ds.write_dataset(lineitem,'./pyarrow', format="parquet")

CPU times: user 2min 27s, sys: 12 s, total: 2min 39s
Wall time: 2min 3s
